In [12]:
from transformers import AutoModel,AutoTokenizer
import torch.nn.functional as F

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased",output_hidden_states=True)

In [14]:
sentence_1="dead"
sentence_2="born"

token_1=tokenizer("dead",return_tensors="pt")
token_2=tokenizer("born",return_tensors="pt")

sentence_1_embedding=model(**token_1).hidden_states
sentence_2_embedding=model(**token_2).hidden_states


In [15]:
vector_1=sentence_1_embedding[12][0][0].unsqueeze(0)
vector_2=sentence_2_embedding[12][0][0].unsqueeze(0)

In [16]:
F.cosine_similarity(vector_1,vector_2)

tensor([0.9177], grad_fn=<SumBackward1>)

In [17]:
from sentence_transformers import SentenceTransformer, util

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

emb1 = sentence_model.encode("dead", convert_to_tensor=True)
emb2 = sentence_model.encode("born", convert_to_tensor=True)

util.cos_sim(emb1, emb2)


tensor([[0.4146]])

In [18]:
#Testing Data
import json 
datas=json.load(open("datas1.json"))
datas

[{'student': 'at sea level 100 celsius degree boils Water.',
  'answer_key': 'Water boils at 100 degree celsius at sea level.'},
 {'student': 'powerhouse cell mitochondria the is of The.',
  'answer_key': 'The mitochondria is the powerhouse of the cell.'},
 {'student': 'law states force upon unless uniform acted object motion an an of external rest in first Newtonâ€™s remains a or.',
  'answer_key': 'Newtonâ€™s first law states that an object remains in a state of rest or uniform motion unless acted upon by an external force.'},
 {'student': 'leaves the chloroplasts happens plants. Photosynthesis the of in in',
  'answer_key': 'Photosynthesis happens in the leaves of plants, in the chloroplasts.'},
 {'student': 'capital Paris. of The France is',
  'answer_key': 'The capital of France is Paris.'},
 {'student': 'change of velocity time. with Acceleration respect is to of rate the',
  'answer_key': 'Acceleration is the rate of change of velocity with respect to time.'},
 {'student': 'sola

In [19]:
def bert_base_eval(datas):
    for data in datas:
        tok_1=tokenizer(data["answer_key"],return_tensors="pt")
        tok_2=tokenizer(data["student"],return_tensors="pt")
        
        sen_1=model(**tok_1).hidden_states[12][0][0]
        sen_2=model(**tok_2).hidden_states[12][0][0]
        similarity_score=F.cosine_similarity(sen_1.unsqueeze(0),sen_2.unsqueeze(0))
        data["result"]=similarity_score[0].item()
        print(similarity_score[0].item())
    return datas
datas=bert_base_eval(datas)

0.8676475882530212
0.8583948612213135
0.8573887348175049
0.8058281540870667
0.7572724223136902
0.7925795912742615
0.8996734619140625
0.8967288732528687


In [20]:
json.dump(datas,open("result.json","w"))

In [21]:
def miniLM_eval(datas):
    for data in datas:
        emb1 = sentence_model.encode(data["student"], convert_to_tensor=True)
        emb2 = sentence_model.encode(data['answer_key'], convert_to_tensor=True)
        print(util.cos_sim(emb1, emb2).item())
miniLM_eval(datas)

0.971549391746521
0.9411647319793701
0.8498973846435547
0.9112030863761902
0.9515818357467651
0.8504413366317749
0.9069758653640747
0.9461259841918945


In [22]:
bert_base_eval(json.load(open("datas2.json",'r')))

0.929330587387085
0.8221029043197632
0.713216245174408
0.892433226108551
0.8916035294532776
0.7825384736061096
0.8471053242683411
0.8463495373725891


[{'student': 'Water boils 100Â°C sea level',
  'answer_key': 'Water boils at 100 degree celsius at sea level.',
  'result': 0.929330587387085},
 {'student': 'Mitochondria powerhouse cell',
  'answer_key': 'The mitochondria is the powerhouse of the cell.',
  'result': 0.8221029043197632},
 {'student': 'Newton first law rest motion external force',
  'answer_key': 'Newtonâ€™s first law states that an object remains in a state of rest or uniform motion unless acted upon by an external force.',
  'result': 0.713216245174408},
 {'student': 'Photosynthesis leaves plants chloroplasts',
  'answer_key': 'Photosynthesis happens in the leaves of plants, in the chloroplasts.',
  'result': 0.892433226108551},
 {'student': 'Capital France Paris',
  'answer_key': 'The capital of France is Paris.',
  'result': 0.8916035294532776},
 {'student': 'Acceleration rate change velocity time',
  'answer_key': 'Acceleration is the rate of change of velocity with respect to time.',
  'result': 0.7825384736061096